# Second Tab Preview (Input Sheet)
Loads only the **2nd tab** from the input Google Sheet and shows it as a pandas DataFrame plus metadata.

In [3]:
import os
from typing import Any, Dict, List, Tuple

import pandas as pd
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

SOURCE_SPREADSHEET_ID = os.environ.get(
    "INPUT_SPREADSHEET_ID",
    "1EXrm0FbudIu44LmgSKXZt2lf96XYdOuwyRWf-6yhXus",
)
CREDENTIALS_PATH = os.environ.get("GOOGLE_APPLICATION_CREDENTIALS", "auto_auth.json")


In [4]:
def _setup_sheets_service():
    if not os.path.isfile(CREDENTIALS_PATH):
        raise FileNotFoundError(
            f"Credentials file not found: {CREDENTIALS_PATH}. Set GOOGLE_APPLICATION_CREDENTIALS or place auto_auth.json in project root."
        )
    creds = Credentials.from_service_account_file(
        CREDENTIALS_PATH,
        scopes=["https://www.googleapis.com/auth/spreadsheets.readonly"],
    )
    return build("sheets", "v4", credentials=creds)


def _normalize_headers(raw_headers: List[Any], width: int) -> List[str]:
    out: List[str] = []
    used: Dict[str, int] = {}
    for i in range(width):
        h = str(raw_headers[i]).strip() if i < len(raw_headers) else ""
        if not h:
            h = f"Column_{i + 1}"
        base = h
        n = used.get(base, 0) + 1
        used[base] = n
        if n > 1:
            h = f"{base}_{n}"
        out.append(h)
    return out


def load_second_tab_df() -> Tuple[pd.DataFrame, Dict[str, Any]]:
    service = _setup_sheets_service()

    meta = service.spreadsheets().get(spreadsheetId=SOURCE_SPREADSHEET_ID).execute()
    sheets = meta.get("sheets", [])
    if len(sheets) < 2:
        raise ValueError(f"Spreadsheet has {len(sheets)} tab(s); cannot load 2nd tab.")

    tab_props = sheets[1].get("properties", {})
    tab_name = tab_props.get("title", "")
    tab_id = tab_props.get("sheetId")
    tab_index = tab_props.get("index", 1)

    range_name = f"'{tab_name}'!A:Z"
    resp = (
        service.spreadsheets()
        .values()
        .get(
            spreadsheetId=SOURCE_SPREADSHEET_ID,
            range=range_name,
            valueRenderOption="UNFORMATTED_VALUE",
            dateTimeRenderOption="SERIAL_NUMBER",
        )
        .execute()
    )
    values = resp.get("values", [])

    if not values:
        df = pd.DataFrame()
        details = {
            "spreadsheet_id": SOURCE_SPREADSHEET_ID,
            "tab_name": tab_name,
            "tab_id": tab_id,
            "tab_index": tab_index,
            "rows_total": 0,
            "columns_total": 0,
            "note": "2nd tab is empty",
        }
        return df, details

    width = max(len(r) for r in values)
    headers = _normalize_headers(values[0], width)

    rows = []
    for r in values[1:]:
        padded = list(r) + [""] * (width - len(r))
        rows.append(padded[:width])

    df = pd.DataFrame(rows, columns=headers)
    non_empty_rows = int((df.astype(str).apply(lambda x: x.str.strip()).ne("").any(axis=1)).sum())

    details = {
        "spreadsheet_id": SOURCE_SPREADSHEET_ID,
        "tab_name": tab_name,
        "tab_id": tab_id,
        "tab_index": tab_index,
        "rows_total_excluding_header": int(df.shape[0]),
        "rows_non_empty_excluding_header": non_empty_rows,
        "columns_total": int(df.shape[1]),
        "columns": list(df.columns),
    }
    return df, details


In [5]:
df, details = load_second_tab_df()
details

{'spreadsheet_id': '1EXrm0FbudIu44LmgSKXZt2lf96XYdOuwyRWf-6yhXus',
 'tab_name': 'January 2026',
 'tab_id': 302677856,
 'tab_index': 1,
 'rows_total_excluding_header': 730,
 'rows_non_empty_excluding_header': 730,
 'columns_total': 13,
 'columns': ['Name',
  'Campaign Title',
  'Type',
  'Type of Compensation',
  'TikTok Profile',
  'Paypal/Crypto',
  'Price per edit',
  '# edits / # views',
  'Total amount (USD)',
  'Genre',
  'Requested by',
  'Payment Status',
  'Payment date']}

In [6]:
df.head(25)

,Name,Campaign Title,Type,Type of Compensation,TikTok Profile,Paypal/Crypto,Price per edit,# edits / # views,Total amount (USD),Genre,Requested by,Payment Status,Payment date
0,em.n.ef,"PHNKR, dnvn, Phonk King - QUANTUM - Slowed",,Flat Fee,https://www.tiktok.com/@em.n.ef,faisal.noelap15@gmail.com,4.5,1,4.50,Phonk,Ninda,PAID,46024
1,vanskennedy,"CryJaxx, MVRPHiN, Lucy Malfroy & Phonk King - ...",,Flat Fee,https://www.tiktok.com/@vanskennedy,sahruldani91dv@gmail.com,4.49,1,4.49,Phonk,Ninda,PAID,46024
2,vanskennedy,"PHNKR, dnvn, Phonk King - QUANTUM - Slowed",,Flat Fee,https://www.tiktok.com/@vanskennedy,sahruldani91dv@gmail.com,4.49,1,4.49,Phonk,Ninda,PAID,46024
3,raaxz_ae,"CryJaxx, MVRPHiN, Lucy Malfroy & Phonk King - ...",,Flat Fee,https://www.tiktok.com/@raaxz_ae,ahmadnizamramadhan07@gmail.com,4.49,1,4.49,Phonk,Ninda,PAID,46024
4,raaxz_ae,"PHNKR, dnvn, Phonk King - QUANTUM - Slowed",,Flat Fee,https://www.tiktok.com/@raaxz_ae,ahmadnizamramadhan07@gmail.com,4.49,1,4.49,Phonk,Ninda,PAID,46024
5,yustio.cs,"CryJaxx, MVRPHiN, Lucy Malfroy & Phonk King - ...",,Flat Fee,https://www.tiktok.com/@yustio.cs,fathirm590@gmail.com,4.49,1,4.49,Phonk,Ninda,PAID,46024
6,yustio.cs,"PHNKR, dnvn, Phonk King - QUANTUM - Slowed",,Flat Fee,https://www.tiktok.com/@yustio.cs,fathirm590@gmail.com,4.49,1,4.49,Phonk,Ninda,PAID,46024
7,Flexxyaep_,"DJ PESO, Obviousgod, NTPV, Phonk King - MONTAG...",,Flat Fee,https://www.youtube.com/@Flexxyaep_/shorts,FLEXXYCR7@gmail.com,8,2,16.00,Phonk,Sujal,PAID,46024
8,wil_ym25,"CryJaxx, MVRPHiN, Lucy Malfroy & Phonk King - ...",,Flat Fee,https://www.tiktok.com/@wil_ym25,akunpaypal525@gmail.com,4.49,1,4.49,Phonk,Ninda,PAID,46025
9,wil_ym25,"Chilx, Rubikdice, Obviousgod, Phonk King - Lua...",,Flat Fee,https://www.tiktok.com/@wil_ym25,akunpaypal525@gmail.com,4.49,1,4.49,Phonk,Ninda,PAID,46025


In [13]:
# Group by member (Requested by) for quick comparison with aggregated outputs
member_col = "Requested by"
status_col = "Payment Status"
song_col = "Song Title"
profile_col = "TikTok Profile"
videos_col = "# edits / # views"
spent_col = "Total amount (USD)"

required = [member_col, status_col]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(
        f"Missing required columns for grouping: {missing}. Available: {list(df.columns)}"
    )

work = df.copy()
work[member_col] = work[member_col].astype(str).str.strip()
work[status_col] = work[status_col].astype(str).str.strip().str.upper()

paid = work[(work[status_col] == "PAID") & (work[member_col] != "")].copy()

if videos_col in paid.columns:
    paid[videos_col] = pd.to_numeric(paid[videos_col], errors="coerce").fillna(0)
if spent_col in paid.columns:
    paid[spent_col] = pd.to_numeric(paid[spent_col], errors="coerce").fillna(0)

agg_map = {"rows_paid": (member_col, "size")}
if song_col in paid.columns:
    agg_map["songs_unique"] = (song_col, pd.Series.nunique)
if profile_col in paid.columns:
    agg_map["pages_unique"] = (profile_col, pd.Series.nunique)
if videos_col in paid.columns:
    agg_map["videos_sum"] = (videos_col, "sum")
if spent_col in paid.columns:
    agg_map["spent_sum"] = (spent_col, "sum")

member_summary = (
    paid.groupby(member_col, dropna=False)
    .agg(**agg_map)
    .reset_index()
)

if "spent_sum" in member_summary.columns:
    member_summary = member_summary.sort_values(
        ["spent_sum", "rows_paid"], ascending=[False, False]
    )
else:
    member_summary = member_summary.sort_values(["rows_paid"], ascending=[False])

# Overall uniqueness metrics
name_col = "Name"
unique_names = None
unique_name_page_pairs = None
unique_campaigns = None

if name_col in paid.columns:
    name_series = paid[name_col].astype(str).str.strip()
    name_series = name_series.replace("", pd.NA).dropna()
    unique_names = int(name_series.nunique())

if (name_col in paid.columns) and (profile_col in paid.columns):
    combo_df = paid[[name_col, profile_col]].copy()
    combo_df[name_col] = combo_df[name_col].astype(str).str.strip()
    combo_df[profile_col] = combo_df[profile_col].astype(str).str.strip()
    combo_df = combo_df[(combo_df[name_col] != "") & (combo_df[profile_col] != "")]
    unique_name_page_pairs = int(combo_df.drop_duplicates().shape[0])

campaign_col = None
for candidate in ["Campaign Title", "Campaign", "Campaign Name", "Campaign name", "Type"]:
    if candidate in paid.columns:
        campaign_col = candidate
        break

if campaign_col is not None:
    campaign_series = paid[campaign_col].astype(str).str.strip()
    campaign_series = campaign_series.replace("", pd.NA).dropna()
    unique_campaigns = int(campaign_series.nunique())

summary_stats = {
    "tab_name": details.get("tab_name"),
    "rows_total_excluding_header": int(df.shape[0]),
    "rows_paid": int(len(paid)),
    "members_in_paid": int(member_summary.shape[0]),
}

if unique_names is not None:
    summary_stats["unique_names"] = unique_names
if unique_name_page_pairs is not None:
    summary_stats["unique_name_page_pairs"] = unique_name_page_pairs
if unique_campaigns is not None:
    summary_stats["unique_campaigns"] = unique_campaigns

print(summary_stats)

# Per-member uniqueness metrics added to the summary table
if name_col in paid.columns:
    per_name = paid[[member_col, name_col]].copy()
    per_name[name_col] = per_name[name_col].astype(str).str.strip()
    per_name = per_name[per_name[name_col] != ""]
    per_name = per_name.drop_duplicates()
    name_counts = (
        per_name.groupby(member_col)[name_col]
        .nunique()
        .rename("unique_names")
    )
    member_summary = member_summary.merge(name_counts, on=member_col, how="left")
    member_summary["unique_names"] = (
        member_summary["unique_names"].fillna(0).astype(int)
    )

if (name_col in paid.columns) and (profile_col in paid.columns):
    combo = paid[[member_col, name_col, profile_col]].copy()
    combo[name_col] = combo[name_col].astype(str).str.strip()
    combo[profile_col] = combo[profile_col].astype(str).str.strip()
    combo = combo[(combo[name_col] != "") & (combo[profile_col] != "")]
    combo = combo.drop_duplicates()
    combo_counts = combo.groupby(member_col).size().rename(
        "unique_name_page_pairs"
    )
    member_summary = member_summary.merge(combo_counts, on=member_col, how="left")
    member_summary["unique_name_page_pairs"] = (
        member_summary["unique_name_page_pairs"].fillna(0).astype(int)
    )

if campaign_col is not None:
    camp = paid[[member_col, campaign_col]].copy()
    camp[campaign_col] = camp[campaign_col].astype(str).str.strip()
    camp = camp[camp[campaign_col] != ""]
    camp = camp.drop_duplicates()
    camp_counts = (
        camp.groupby(member_col)[campaign_col]
        .nunique()
        .rename("unique_campaigns")
    )
    member_summary = member_summary.merge(camp_counts, on=member_col, how="left")
    member_summary["unique_campaigns"] = (
        member_summary["unique_campaigns"].fillna(0).astype(int)
    )

# Show the output sorted by the request by column
member_summary_sorted = member_summary.sort_values(member_col)
member_summary_sorted.head(100)


{'tab_name': 'January 2026', 'rows_total_excluding_header': 730, 'rows_paid': 730, 'members_in_paid': 11, 'unique_names': 224, 'unique_name_page_pairs': 235, 'unique_campaigns': 39}


,Requested by,rows_paid,pages_unique,videos_sum,spent_sum,unique_names,unique_name_page_pairs,unique_campaigns
2,Amey,100,48,145.0,2260.00,48,48,17
4,Auren,14,13,137.0,1520.00,13,13,2
8,Chris,3,1,9.0,225.00,1,1,3
7,Discord,8,5,11.0,380.59,5,5,4
0,Dominik,186,63,358.0,5898.20,61,63,17
6,Donovan,2,1,2.0,500.00,1,1,2
9,Fyodor,3,3,3.0,50.00,3,3,2
3,Ninda,193,55,198.0,1588.62,54,55,20
10,Quan,3,2,3.0,47.59,2,2,2
1,Rajveer,171,29,275.0,2602.00,23,29,27


In [14]:
# Show unique TikTok Profile + Name pairs for a specific member
member_filter = "Rajveer"  # change this if you want another member

if member_col not in paid.columns:
    raise ValueError(f"Column '{member_col}' not found in paid DataFrame")

if ("Name" not in paid.columns) or (profile_col not in paid.columns):
    raise ValueError("Expected both 'Name' and TikTok profile columns to be present")

subset = paid[paid[member_col] == member_filter].copy()

pairs = subset[["Name", profile_col]].copy()
pairs["Name"] = pairs["Name"].astype(str).str.strip()
pairs[profile_col] = pairs[profile_col].astype(str).str.strip()

pairs = pairs[(pairs["Name"] != "") & (pairs[profile_col] != "")]

unique_pairs = pairs.drop_duplicates().sort_values(["Name", profile_col]).reset_index(drop=True)
unique_pairs

,Name,TikTok Profile
0,_car_edit_4k,https://www.instagram.com/_car_edit_4k?
1,_otaku_ningen_,https://www.instagram.com/_otaku_ningen_?
2,anifixtube,https://www.instagram.com/anifixtube
3,anifixtube,https://www.instagram.com/anifixtube?
4,asad,in house editor
5,carforedit4k,https://www.instagram.com/carforedit4k?
6,dynamic_ae,https://www.instagram.com/dynamic_ae
7,footballmoments.7x,https://www.instagram.com/footballmoments.7x
8,footballmoments.7x,https://www.instagram.com/footballmoments.7x?
9,gif_names_finding,https://www.instagram.com/gif_names_finding?


In [ ]:
# Unique values per column for the selected member
member_filter = "Rajveer"  # must match the member_col value

if member_col not in paid.columns:
    raise ValueError(f"Column '{member_col}' not found in paid DataFrame")

subset = paid[paid[member_col] == member_filter].copy()

if subset.empty:
    raise ValueError(f"No rows found for member '{member_filter}'")

unique_rows = []
for col in subset.columns:
    series = subset[col].astype(str).str.strip()
    series = series[series != ""]
    uniques = sorted(series.unique())
    unique_rows.append({"column": col, "unique_values": uniques})

unique_per_column_df = pd.DataFrame(unique_rows)
unique_per_column_df